In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Create Organized Folder 
import os

raw_path = '/content/drive/MyDrive/body-shape-classification/raw-body-shape-dataset'
organized_path = '/content/drive/MyDrive/body-shape-classification/organized-dataset'

os.makedirs(organized_path, exist_ok=True)

In [ ]:
#Organize Images into Class Folders
import shutil
import os # Import os module here

for file in os.listdir(raw_path):
    if not file.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue
    label = file.split('_')[0]
    label = 'Inverted' if label == 'InvertedTriangle' else label  # normalize
    label_folder = os.path.join(organized_path, label)
    os.makedirs(label_folder, exist_ok=True)

    src = os.path.join(raw_path, file)
    dst = os.path.join(label_folder, file)
    shutil.copy(src, dst) #Copy the image to the right folder based on its label.

In [ ]:
from collections import Counter

counts = Counter({
    folder: len(os.listdir(os.path.join(organized_path, folder)))
    for folder in os.listdir(organized_path)
    if os.path.isdir(os.path.join(organized_path, folder))
})
print("Class distribution:", counts)

# check if your dataset is balanced or imbalanced.

Class distribution: Counter()


In [ ]:
from PIL import Image

def verify_images(directory):
    for folder in os.listdir(directory):
        path = os.path.join(directory, folder)
        for img_file in os.listdir(path):
            img_path = os.path.join(path, img_file)
            try:
                with Image.open(img_path) as img:
                    img.verify()
            except Exception as e:
                print(f"Corrupt: {img_path} — {e}")

verify_images(organized_path)


In [ ]:
from collections import defaultdict

def count_images(path):
    dist = defaultdict(dict)
    for split in splits:
        for cls in os.listdir(os.path.join(path, split)):
            folder = os.path.join(path, split, cls)
            dist[split][cls] = len(os.listdir(folder))
    return dist

counts = count_images(target_base)
for split in counts:
    print(f"{split.upper()}:")
    for cls in counts[split]:
        print(f"  {cls}: {counts[split][cls]}")


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import os

In [ ]:
data_dir = '/content/drive/MyDrive/body-shape-classification/split-dataset'
img_size = (224, 224)  # MobileNetV2 input size
batch_size = 16


In [ ]:
train_datagen = ImageDataGenerator( #data augmentation
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    os.path.join(data_dir, 'train'),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_gen = val_test_datagen.flow_from_directory(
    os.path.join(data_dir, 'val'),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_gen = val_test_datagen.flow_from_directory(
    os.path.join(data_dir, 'test'),
    target_size=img_size,
    batch_size=1,
    class_mode='categorical',
    shuffle=False
)


In [ ]:
#Load MobileNetV2 pre-trained on ImageNet, without top layers 
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
#Freeze the base model to retain learned features
base_model.trainable = False  # Freeze base for now

x = base_model.output
x = GlobalAveragePooling2D()(x)
output = Dense(5, activation='softmax')(x)  # assuming 5 classes 

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(learning_rate=0.0005),#Compile the model
              loss='categorical_crossentropy',
              metrics=['accuracy'])


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
from sklearn.utils import class_weight
import numpy as np

# Get class indices from train generator
labels = train_gen.classes
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(labels),
    y=labels
)
class_weights = dict(enumerate(class_weights))
print("Class Weights:", class_weights)


history = model.fit(
    train_gen,
    epochs=10,
    validation_data=val_gen,
    class_weight=class_weights
)


Class Weights: {0: np.float64(3.7466666666666666), 1: np.float64(0.5017857142857143), 2: np.float64(1.146938775510204), 3: np.float64(0.6386363636363637), 4: np.float64(3.3058823529411763)}


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 58s 3s/step - accuracy: 0.2519 - loss: 1.8992 - val_accuracy: 0.1544 - val_loss: 1.8233
Epoch 2/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - accuracy: 0.2104 - loss: 1.7557 - val_accuracy: 0.3826 - val_loss: 1.4669
Epoch 3/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.2747 - loss: 1.5555 - val_accuracy: 0.4161 - val_loss: 1.4445
Epoch 4/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.3291 - loss: 1.4259 - val_accuracy: 0.4362 - val_loss: 1.4211
Epoch 5/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - accuracy: 0.3851 - loss: 1.3514 - val_accuracy: 0.4899 - val_loss: 1.3229
Epoch 6/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.4099 - loss: 1.2976 - val_accuracy: 0.4966 - val_loss: 1.2840
Epoch 7/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - accuracy: 0.3891 - loss: 1.3323 - val_accuracy: 0.5570 - val_loss: 1.1829
Epoch 8/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.4715 - loss: 1.2745 - val_accuracy: 0.5436 - val_loss:

In [ ]:
# UNFREEZE the base model
base_model.trainable = True

# Compile again with a lower learning rate
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train again with class weights
model.fit(
    train_gen,
    epochs=10,
    validation_data=val_gen,
    class_weight=class_weights
)


Epoch 1/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 95s 3s/step - accuracy: 0.3513 - loss: 1.4207 - val_accuracy: 0.5973 - val_loss: 1.0871
Epoch 2/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 58s 3s/step - accuracy: 0.3623 - loss: 1.3176 - val_accuracy: 0.6174 - val_loss: 1.0613
Epoch 3/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 64s 4s/step - accuracy: 0.3949 - loss: 1.4021 - val_accuracy: 0.6242 - val_loss: 1.0528
Epoch 4/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 58s 3s/step - accuracy: 0.4297 - loss: 1.1986 - val_accuracy: 0.6577 - val_loss: 1.0427
Epoch 5/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 58s 3s/step - accuracy: 0.4646 - loss: 1.1620 - val_accuracy: 0.6644 - val_loss: 1.0352
Epoch 6/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 58s 3s/step - accuracy: 0.4317 - loss: 1.1211 - val_accuracy: 0.6577 - val_loss: 1.0255
Epoch 7/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 60s 3s/step - accuracy: 0.4574 - loss: 1.1117 - val_accuracy: 0.6711 - val_loss: 1.0094
Epoch 8/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 58s 3s/step - accuracy: 0.5436 - loss: 0.9715 - val_accuracy: 0.6779 - val_loss:

In [ ]:
loss, accuracy = model.evaluate(test_gen)
print(f"Test Accuracy: {accuracy:.4f}")


172/172 ━━━━━━━━━━━━━━━━━━━━ 31s 182ms/step - accuracy: 0.6303 - loss: 1.0013
Test Accuracy: 0.6279


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

preds = model.predict(test_gen)
y_pred = np.argmax(preds, axis=1)
y_true = test_gen.classes
labels = list(test_gen.class_indices.keys())

print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=labels))


172/172 ━━━━━━━━━━━━━━━━━━━━ 9s 42ms/step
[[10  0  0  0  1]
 [ 3 32  9 17  5]
 [ 1  3 18  7  2]
 [ 0  5  6 37  5]
 [ 0  0  0  0 11]]
              precision    recall  f1-score   support

       Apple       0.71      0.91      0.80        11
   Hourglass       0.80      0.48      0.60        66
    Inverted       0.55      0.58      0.56        31
   Rectangle       0.61      0.70      0.65        53
    Triangle       0.46      1.00      0.63        11

    accuracy                           0.63       172
   macro avg       0.62      0.73      0.65       172
weighted avg       0.67      0.63      0.62       172



In [ ]:
model.save("body_shape_keras_model.h5")

In [ ]:
model.save("body_shape_keras_model.keras")